 # Solution 2: Inference FinBERT

FinBERT es un modelo de análisis de sentimiento de textos financieros basado en BERT.

Se basa en BERT, y se ha adaptado al entorno financiero, usando un corpus legal y luego fine-tuning. Detalles en: https://github.com/ProsusAI/finBERT

IMPORTANTE: la implementación se basa en el modelo BERT disponible en Hugging Face's pytorch_pretrained_bert, que usa una versión antigua de la librería transformers.

El ejemplo de abajo sube los experimentos a wandb.

In [ ]:
#!pip install transformers
#!pip install wandb

In [ ]:
import pandas

headlines_df = pandas.read_csv('test.csv')
headlines_df.head(5)

In [ ]:
import numpy as np

headlines_array = np.array(headlines_df)
np.random.shuffle(headlines_array)
headlines_list = list(headlines_array[:,2])

print(headlines_list)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [ ]:
#That's where the headlines get tokenized to be inputted into model

inputs = tokenizer(headlines_list, padding = True, truncation = True, return_tensors='pt')
print(inputs)

In [ ]:
#inference

outputs = model(**inputs)
print(outputs.logits.shape)

In [ ]:
# Postprocessing with softmax

import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

In [ ]:
#Model classes

model.config.id2label

In [ ]:
#Formatting the results as a pandas data frame

import pandas as pd

#Headline #Positive #Negative #Neutral
positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()

table = {'Headline':headlines_list,
         "Positive":positive,
         "Negative":negative,
         "Neutral":neutral}

df = pd.DataFrame(table, columns = ["Headline", "Positive", "Negative", "Neutral"])

df.head(5)


In [ ]:
#Logging the pandas table as a WandB Table

import wandb

wandb.init(project="finbert-sentiment-analysis")
wandb.run.log({"Financial Sentiment Analysis Table" : wandb.Table(dataframe=df)})
wandb.run.finish()